### **ECON 323 FINAL PROJECT**

### **PARLIAMENTARY PARTICIPATION AND ELECTORAL SUCCESS**

##### INTRODUCTION

In this project, we aim to create a visualization notebook based on a ECON 490 paper. The paper explores the impact of parliamentary participation on electoral success particularly in the Canadian House of Commons. 

In this case, parliamentary participation is measured by participation in question period and statements made in the House of Commons and the private member bills introduced by the Member of Parliament. Electoral success is measured by whether or not the Member of Parliament was reelected in the next federal elections.

For this project, we will create a visualization notebook that focuses on:
- visually summarizing the variables 
- exploring relationships between the independent and dependent variables 

##### LOADING PACKAGES

We start by loading all the necessary packages:

In [ ]:
import pandas as pd
import numpy as np
import shapefile as shp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import geopandas as gpd
import matplotlib.colors as colors
import folium 
from folium import plugins

##### INTRODUCING THE DATA 

##### *Reading the Data*

We will first load in the data from csv files and rename columns so that the two dataframes can be concatenated.

The data in the csv files has been compiled from several sources:
- Elections Canada for reelection and riding information
- OurCommons website for information on Private Member's Bills, statements made and Cabinet ministers
- SQL Database from the openparliament.com website for gender, party and politician name
- Conservative.ca website for information on Shadow Cabinet
- Age and years in House of Commons from Parl.ca website 

The timeframe for this data is from the 2015 federal elections to the 2021 federal elections i.e. the information considers all parliamentary activity from the 42nd and 43rd Parliament.

In [ ]:
# reading in csv data 
results15 = pd.read_csv("DATA/2015Results.csv")
results19 = pd.read_csv("DATA/2019Results.csv")

In [ ]:
# renaming columns so that dataframes can be concatenated correctly 
Results15 = results15.rename(columns = {"RidingID":"Riding ID", "ranfor43":"RanForNE", "reelected43":"Reelected", "DOB1":"DOB", "YearsServedAtTimeOfElection":"YearsInHOC"})
Results19 = results19.rename(columns = {"Election":"Parliament", "ranfor44":"RanForNE", "reelected44":"Reelected", "StatementsByMembers":"Statements", "YearsServedAtElection":"YearsInHOC"})

In [ ]:
# concatenating dataframes
Results_Full = pd.concat([Results15, Results19], ignore_index=True)
# combining 2015 and 2019 datasets leds to MPs showing up twice -- doesn't matter for analysis because each row is just one observation - person itself doesnt matter

##### *Creating Dummy Variables*

Next, we create dummy variables for the following categorical variables:
- Gender
- RanForNE (i.e. whether or not the MP ran for the next federal election)
- ReelectionNE (i.e. whether or not the MP was reelected in the next federal election)
- Party (i.e. whether or not a MP belong to the Conservative Party, Liberal Party or the New Democratic Party)

In [ ]:
# creating a dataframe with columns that need to be converted to dummy variables
Results_Dummy = pd.DataFrame(data=Results_Full, columns=['Gender', 'RanForNE', 'Reelected', 'Party'])

# creating dummy variables for each column of the above dataset
Dummies = pd.get_dummies(Results_Dummy)

Then, we add the previously created dummy variables to the original dataframe. Just to clean up, we rename some columns for clarity and drop unnecessary columns.

In [ ]:
# adding newly created dummy variables to the original dataframe
Results = pd.concat([Results_Full, Dummies], axis = 1)

# removing unnecessary columns
Results = Results.drop(columns=['Gender', 'RanForNE', 'Reelected', 'Party', 'DOB', 'Gender_M', 'RanForNE_No', 'Reelected_.', 'Reelected_No', 'Party_Bloc Québécois', 'Party_Green Party of Canada', 'Party_Independent'])

# renaming columns for better understanding
Results = Results.rename(columns = {"Gender_F":"Gender", "RanForNE_Yes":"RanForNE", "Reelected_Yes":"Reelected", "Party_Conservative Party of Canada":"ConservativeMem", "Party_Liberal Party of Canada":"LiberalMem", "Party_New Democratic Party":"NDPMem"})

##### *Manipulating the Data*

- create a histogram here? to explain questionlog 

In [ ]:
# creating a log of the QuestionPeriod variable
Results['QuestionsLog'] = np.log10(Results['QuestionPeriod'])

# removing all observations where questions asked is 0
Results = Results.loc[Results['QuestionPeriod'] >= 1]

Now just summarize the data - where we are at so far basically

In [ ]:
PlottingDF = pd.DataFrame(data=Results, columns=['Name', 'Parliament', 'Riding ID', 'RanForNE', 'Reelected'])

def f(row):
    if row['RanForNE'] == row['Reelected'] == 1:
        val = 'Ran for Re-election and Won'
    elif row['RanForNE'] == 0:
        val = 'Did Not Run for Re-election'
    else:
        val = 'Ran for Re-election and Lost'
    return val

PlottingDF['Outcome'] = PlottingDF.apply(f, axis=1)

Plotting19 = PlottingDF.loc[PlottingDF['Parliament'] == 42] # results for 2019 
Plotting21 = PlottingDF.loc[PlottingDF['Parliament'] == 43] # results for 2021

##### VISUALIZATION: VARIABLES

##### *Reelection*

##### VISUALIZATION: RELATIONSHIPS

##### *Questions and Gender*

In [ ]:
# loading in the shapefile
Ridings = gpd.read_file('FED_CA_2021_EN/FED_CA_2021_EN.shp')

# renaming the column so that the dataframes can be merged based on Riding ID
Ridings = Ridings.rename(columns = {"FED_NUM":"Riding ID", "ED_NAMEE":"Electoral District"})

https://open.canada.ca/data/en/dataset/47a0f098-7445-41bb-a147-41686b692887 - SOURCE FOR SHAPEFILE

In [ ]:
# merging so that we have an outcome for each electoral riding
map2019 = pd.merge(Ridings, Plotting19, on = 'Riding ID')
map2021 = pd.merge(Ridings, Plotting21, on = 'Riding ID')

In [ ]:
map2019.explore(
     column="Outcome",
     tooltip="Outcome",
     popup = ['Name', 'Electoral District', 'Outcome'],
     cmap="Paired",
     style_kwds=dict(color="black") 
    )

In [ ]:
map2021.explore(
     column="Outcome",
     tooltip="Outcome",
     popup = ['Name', 'Electoral District', 'Outcome'],
     cmap="Paired",
     style_kwds=dict(color="black") 
    )

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20))
map2019.plot(ax=ax, column='Outcome', legend=True, edgecolor='k', cmap = 'Paired')
plt.axis('off')
plt.show()